# <center> Data cleaning
    
    

Read files from ETL notebook

In [1]:
%load_ext jupyternotify

%store -r item_cat
%store -r item
%store -r sub
%store -r shops
%store -r sales_test
%store -r sales_train

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [2]:
import sys
import os
sys.path.append(os.path.realpath('./../../scripts'))
sys.path = list(set(sys.path))

In [3]:
__da

/Users/maksymsuprunenko/anaconda3/lib/python3.7/site-packages/tqdm/_tqdm.py:604: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Basic Data Analysis tools was loaded


In [4]:
from features_transformers import DateFeatureExtractor

In [34]:
sales_train_cleaned = sales_train.copy()

In [35]:
#%%time
duplicated_cols = sales_train.groupby(['date', 'item_id', 'shop_id'])['date_block_num'].count().reset_index()
duplicated_cols = duplicated_cols[duplicated_cols['date_block_num'] == 1].rename(columns={'date_block_num': 'count'})
duplicated_cols = sales_train.groupby(['date', 'item_id', 'shop_id'])['date_block_num'].count().reset_index()
duplicated_cols = duplicated_cols[duplicated_cols['date_block_num'] > 1]
duplicated_cols = duplicated_cols.reset_index(drop=True) 

sales_train_cleaned = sales_train_cleaned.drop_duplicates(['date', 'item_id', 'shop_id'])
sales_train.shape[0] - sales_train_cleaned.shape[0]

28

One of the items has price -1. Change this item price to median value.

In [7]:
sales_train_cleaned[sales_train_cleaned['item_price'] < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,2013-05-15,4,32,2973,-1.00,1.00


In [36]:
median_2973_item_price = sales_train_cleaned['item_price'][sales_train_cleaned['item_id']  == 2934].median()
sales_train_cleaned['item_price'] = sales_train_cleaned["item_price"].replace(-1, value=median_2973_item_price)

assert sales_train_cleaned.loc[np.where(sales_train_cleaned['item_price'] < 0)[0], 'item_price'].shape[0] == 0

In [57]:
train = sales_train_cleaned

In [97]:
%%notify

from itertools import product
import time

train = sales_train_cleaned
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in tqdm(range(34)):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

train['revenue'] = train['item_price'] *  train['item_cnt_day']

ts = time.time()
group = train.groupby(['date_block_num','shop_id','item_id'])\
    .agg({'item_cnt_day': ['sum'], 'item_price': ['median']})

group.columns = ['item_cnt_month', 'item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float32))

matrix['item_price'] = (matrix['item_price']
                                .fillna(0)
                                .astype(np.float32))


100%|██████████| 34/34 [00:06<00:00,  5.52it/s]


<IPython.core.display.Javascript object>

In [98]:
train = matrix[matrix['item_price'] < 100000]
train = train[matrix['item_cnt_month'] < 1010]

/Users/maksymsuprunenko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [100]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [107]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [111]:
ts = time.time()

train_test = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
train_test.fillna(0, inplace=True) 

time.time() - ts

0.3145570755004883

In [113]:
train_test = train_test.drop('ID', axis=1)

In [118]:
%store train
%store train_test

Stored 'train' (DataFrame)
Stored 'train_test' (DataFrame)


In [115]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10913849 entries, 0 to 10913849
Data columns (total 5 columns):
date_block_num    int8
shop_id           int8
item_id           int16
item_cnt_month    float32
item_price        float32
dtypes: float32(2), int16(1), int8(2)
memory usage: 208.2 MB


In [116]:
train_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128050 entries, 0 to 11128049
Data columns (total 5 columns):
date_block_num    int8
shop_id           int8
item_id           int16
item_cnt_month    float32
item_price        float32
dtypes: float32(2), int16(1), int8(2)
memory usage: 127.4 MB


In [117]:
import gc
gc.collect()

181